In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [2]:
# Load the data into a Pandas dataframe
df = pd.read_csv("train_transaction.csv")

In [3]:
# Split the data into numerical and categorical features
num_features = df.select_dtypes(include=[np.number]).columns.tolist()
num_features.remove("isFraud")
cat_features = df.select_dtypes(include=[object]).columns.tolist()
print(1)
# Impute missing values for numerical features with the mean
num_imputer = SimpleImputer(strategy='mean')
df[num_features] = num_imputer.fit_transform(df[num_features])
print(2)
# Impute missing values for categorical features with the most frequent value
cat_imputer = SimpleImputer(strategy='most_frequent')
df[cat_features] = cat_imputer.fit_transform(df[cat_features])
print(3)

1
2
3


In [4]:
df.describe()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
count,5.905400e+05,590540.000000,5.905400e+05,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,...,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000
mean,3.282270e+06,0.034990,7.372311e+06,135.027176,9898.734658,362.555488,153.194925,199.278897,290.733794,86.800630,...,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.164550,28.530903,55.352422,151.160542,100.700882
std,1.704744e+05,0.183755,4.617224e+06,239.162522,4901.170153,156.595240,11.321413,41.095455,95.914128,2.536525,...,1.765559,2321.688741,4170.926347,2970.898706,91.064804,144.751912,102.534859,249.632064,408.917095,304.324367
min,2.987000e+06,0.000000,8.640000e+04,0.251000,1000.000000,100.000000,100.000000,100.000000,100.000000,10.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.134635e+06,0.000000,3.027058e+06,43.321000,6019.000000,215.000000,150.000000,166.000000,205.000000,87.000000,...,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.164550,28.530903,55.352422,151.160542,100.700882
50%,3.282270e+06,0.000000,7.306528e+06,68.769000,9678.000000,361.000000,150.000000,226.000000,290.733794,87.000000,...,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.164550,28.530903,55.352422,151.160542,100.700882
75%,3.429904e+06,0.000000,1.124662e+07,125.000000,14184.000000,512.000000,150.000000,226.000000,327.000000,87.000000,...,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.164550,28.530903,55.352422,151.160542,100.700882
max,3.577539e+06,1.000000,1.581113e+07,31937.391000,18396.000000,600.000000,231.000000,237.000000,540.000000,102.000000,...,55.000000,160000.000000,160000.000000,160000.000000,55125.000000,55125.000000,55125.000000,104060.000000,104060.000000,104060.000000


In [5]:
df.shape

(590540, 394)

In [6]:
# Preprocess the numerical features
scaler = StandardScaler()
X_num = scaler.fit_transform(df[num_features])

# Preprocess the categorical features
encoder = OneHotEncoder()
X_cat = encoder.fit_transform(df[cat_features])

In [11]:
# Combine the numerical and categorical features
X = pd.concat([pd.DataFrame(X_num), pd.DataFrame(X_cat.toarray())], axis=1)
y = df["isFraud"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Train a Logistic Regression model
classifier = LogisticRegression(max_iter=100)
classifier.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [12]:
X.shape

(590540, 530)

In [13]:
# Predict the class labels for the test set
y_pred = classifier.predict(X_test)

In [14]:
# Evaluate the model's performance
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99    114056
           1       0.81      0.26      0.39      4052

    accuracy                           0.97    118108
   macro avg       0.89      0.63      0.69    118108
weighted avg       0.97      0.97      0.97    118108

